In [1]:
import os
from dotenv import load_dotenv, find_dotenv
result = load_dotenv(find_dotenv()) # read local .env file

print (os.environ["OPENAI_API_BASE"])
langchain_tracing_v2 = os.environ.pop("LANGCHAIN_TRACING_V2", None)
langchain_endpoint = os.environ.pop("LANGCHAIN_ENDPOINT", None)
langchain_api_key = os.environ.pop("LANGCHAIN_API_KEY", None)
langchain_project = os.environ.pop("LANGCHAIN_PROJECT", None)



https://alkopenai2.openai.azure.com/


## Base Invocation

Base invocation allows you to simply call the LLM without worrying about anyting. Just place your prompt and go on.

In [2]:
from langchain import PromptTemplate
from langchain_openai  import AzureChatOpenAI
from langchain import LLMChain

# Remember that the deployment name can be different from the model name
# and that you need to create a .env with the OPENAI_API_KEY variable
#and OPENAI_API_BASE
llm = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    deployment_name="GPT-4o-mini", #Deployment name
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    model_name="GPT-4o-mini"
)

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# deprecated
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

# user question
question = "Who is leonardo da vinci? Answer like capitan barbossa."
result = llm_chain.run(question)
print(result)


/var/folders/5_/w60z7jfd3kggxf5884bbg2ym0000gn/T/ipykernel_48292/3208901216.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/var/folders/5_/w60z7jfd3kggxf5884bbg2ym0000gn/T/ipykernel_48292/3208901216.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run(question)


Ahoy there! Leonardo da Vinci, ye see, be a master of many trades, a true Renaissance man, if ye will. Born in the year of our Lord 1452 in the land of Italy, he be not just an artist, nay! He be a thinker, a scientist, a tinkerer of contraptions that'd make yer head spin faster than a ship caught in a maelstrom!

With brush in hand, he painted the likes of the Mona Lisa, a lass with a smile that could charm the very sea itself. He sketched machines that could take flight, long before the likes of us ever dared to dream of soaring through the skies! Aye, he studied the stars, the human form, and the world around him like a captain charts the course of his ship.

So, if ye be lookin' for brilliance, look no further than this fella, for he be a legend carved into the annals of time, a true treasure of knowledge and creativity! Savvy?


In [ ]:
invocation = prompt | llm
response = invocation.invoke({"question": question})
print(response.content)

# Batch capabilities

If you need to have multiple prompts, you can simply pass a list of dictionaries to the `generate` method.


In [ ]:
from pprint import pprint
qs = [
    {'question': "Who is leonardo da vinci? Answer like capitan barbossa."},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
]
batch_result = llm_chain.generate(qs)

# Loop through the results and print each question and its corresponding answer
for i, res in enumerate(batch_result.generations):
    print(f"Question {i+1}: {qs[i]['question']}")
    print(f"Answer {i+1}: {res[0].text}")
    print("\n")


In [ ]:
# Print all properties of the first result to show its shape, do a pretty print
first_result = batch_result.generations[0]
first_answer = first_result[0]
print(f"The type of batch_result is: {type(batch_result)}")
print(f"The type of batch_result.generations is: {type(batch_result.generations)}")
print(f"The type of first_result is: {type(first_result)}")
print(f"The type of first_answer is: {type(first_answer)}")

pprint(vars(first_answer))


In [ ]:
pprint(vars(first_answer.message))

In [ ]:
response_metadata = first_answer.message.response_metadata

print("Content Filter Results:")
print(f"  Hate: Filtered: {response_metadata['content_filter_results']['hate']['filtered']}, Severity: {response_metadata['content_filter_results']['hate']['severity']}")
print(f"  Protected Material Code: Detected: {response_metadata['content_filter_results']['protected_material_code']['detected']}, Filtered: {response_metadata['content_filter_results']['protected_material_code']['filtered']}")
print(f"  Protected Material Text: Detected: {response_metadata['content_filter_results']['protected_material_text']['detected']}, Filtered: {response_metadata['content_filter_results']['protected_material_text']['filtered']}")
print(f"  Self Harm: Filtered: {response_metadata['content_filter_results']['self_harm']['filtered']}, Severity: {response_metadata['content_filter_results']['self_harm']['severity']}")
print(f"  Sexual: Filtered: {response_metadata['content_filter_results']['sexual']['filtered']}, Severity: {response_metadata['content_filter_results']['sexual']['severity']}")
print(f"  Violence: Filtered: {response_metadata['content_filter_results']['violence']['filtered']}, Severity: {response_metadata['content_filter_results']['violence']['severity']}")

print(f"Finish Reason: {response_metadata['finish_reason']}")
print(f"Logprobs: {response_metadata['logprobs']}")
print(f"Model Name: {response_metadata['model_name']}")

print("Prompt Filter Results:")
for prompt_filter in response_metadata['prompt_filter_results']:
    print(f"  Prompt Index: {prompt_filter['prompt_index']}")
    print(f"  Hate: Filtered: {prompt_filter['content_filter_results']['hate']['filtered']}, Severity: {prompt_filter['content_filter_results']['hate']['severity']}")
    print(f"  Jailbreak: Detected: {prompt_filter['content_filter_results']['jailbreak']['detected']}, Filtered: {prompt_filter['content_filter_results']['jailbreak']['filtered']}")
    print(f"  Self Harm: Filtered: {prompt_filter['content_filter_results']['self_harm']['filtered']}, Severity: {prompt_filter['content_filter_results']['self_harm']['severity']}")
    print(f"  Sexual: Filtered: {prompt_filter['content_filter_results']['sexual']['filtered']}, Severity: {prompt_filter['content_filter_results']['sexual']['severity']}")
    print(f"  Violence: Filtered: {prompt_filter['content_filter_results']['violence']['filtered']}, Severity: {prompt_filter['content_filter_results']['violence']['severity']}")

print(f"System Fingerprint: {response_metadata['system_fingerprint']}")

print("Token Usage:")
print(f"  Completion Tokens: {response_metadata['token_usage']['completion_tokens']}")
print(f"  Completion Tokens Details: {response_metadata['token_usage']['completion_tokens_details']}")
print(f"  Prompt Tokens: {response_metadata['token_usage']['prompt_tokens']}")
print(f"  Prompt Tokens Details: {response_metadata['token_usage']['prompt_tokens_details']}")
print(f"  Total Tokens: {response_metadata['token_usage']['total_tokens']}")

You can instead simply create a single prompt, but ask the LLM to answer all the questions one at a time


In [ ]:
multi_template = """Answer the following questions one at a time.
Separate each new answer with ----

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = long_prompt | llm

qs_str =  "Who is leonardo da vinci? Answer like capitan barbossa.\n" + \
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" + \
    "Who was the 12th person on the moon?" 

answer = llm_chain.invoke(qs_str)
print(answer.content)


In [ ]:
response_metadata = answer.response_metadata

print("Token Usage:")
print(f"  Completion Tokens: {response_metadata['token_usage']['completion_tokens']}")
print(f"  Completion Tokens Details: {response_metadata['token_usage']['completion_tokens_details']}")
print(f"  Prompt Tokens: {response_metadata['token_usage']['prompt_tokens']}")
print(f"  Prompt Tokens Details: {response_metadata['token_usage']['prompt_tokens_details']}")
print(f"  Total Tokens: {response_metadata['token_usage']['total_tokens']}")


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_community.callbacks import get_openai_callback

def process_prompt(prompt: str):
    model = llm
    response = model.invoke(prompt)
    return response

In [ ]:

from langchain_community.callbacks.openai_info import OpenAICallbackHandler

with get_openai_callback() as cb:
    output = RunnableLambda(process_prompt).invoke("Who is leonardo da vinci? Answer like capitan barbossa.")

print("Output:", output.content)
attributes = [
    'always_verbose', 'completion_tokens', 'ignore_agent', 'ignore_chain', 'ignore_chat_model', 
    'ignore_custom_event', 'ignore_llm', 'ignore_retriever', 'ignore_retry', 'prompt_tokens', 'raise_error', 
    'run_inline', 'successful_requests', 'total_cost', 'total_tokens'
]
print('\n')
formatted_total_cost = f"${cb.total_cost:.8f}"
print(f"total_cost: {formatted_total_cost}")

for attr in attributes:
    print(f"{attr}: {getattr(cb, attr)}")
#for attr in dir(cb):
#    if not attr.startswith("__"):
#        print(f"{attr}: {getattr(cb, attr)}")
